In [4]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression

# Para separar el el dataframe en test de validación y test de entrenamiento
from sklearn.model_selection import train_test_split
# Para hacer varias evaluaciones del modelo con un conjunto de validación 
from sklearn.model_selection import cross_val_score
# Métricas
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
# Modelos
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC

from nltk import word_tokenize

In [5]:
# Cargar los datos
df_train = pd.read_excel(r'C:\Users\efreay\Documents\Academia\correos.xlsx')
df_val = pd.read_excel(r'C:\Users\efreay\Documents\Academia\nuevos_correos.xlsx')

df_val["Subject"] = df_val["Subject"].astype(str)
df_val["Preview"] = df_val["Preview"].astype(str)

In [6]:
# Convertir la columna 'DateTimeSent' a tipo datetime
df_val['DateTimeSent'] = pd.to_datetime(df_val['DateTimeSent'])

# Obtener el mes actual
mes_actual = pd.to_datetime('today').month
print(mes_actual)

10


In [7]:
#  Eliminar las filas que no corresponden al mes actual
df_val = df_val[~(df_val['DateTimeSent'].dt.month != mes_actual)]

In [8]:
# Crear una tupla con las palabras o frases a eliminar
palabras_a_eliminar = ("Solicitud atendida", "Notificación Finalizaciòn", "RPA", 
                       "El buzón está lleno.")

# Crear una función para determinar si una fila debe eliminarse
def debe_eliminar(fila):
    for palabra in palabras_a_eliminar:
        if palabra in fila["Subject"]:
            return True
    return False

# Eliminar las filas que cumplan con el criterio
df_val = df_val[~df_val.apply(debe_eliminar, axis=1)]

In [9]:
# Define una función para transformar la columna "Clasificación"
def transform_clasificacion(clasificacion):
    if clasificacion == "Certificación Bancaria Basica":
        return clasificacion
    else:
        return "otros"
    
# Aplica la función a la columna "Clasificación" y crea una nueva columna "Clasificación Nueva"
df_train['Clasificación'] = df_train['Clasificación'].apply(transform_clasificacion)

In [10]:
# Normalizar el texto
def normalize_text(text):
    text = text.lower()
    text = text.replace("á", "a")
    text = text.replace("é", "e")
    text = text.replace("í", "i")
    text = text.replace("ó", "o")
    text = text.replace("ú", "u")

    return text

In [11]:
# Aplicar la normalización al dataset de entrenamiento
df_train["Subject"] = df_train["Subject"].apply(normalize_text)
df_train["Preview"] = df_train["Preview"].apply(normalize_text)

# Aplicar la normalización al dataset de validación
df_val["Subject"] = df_val["Subject"].apply(normalize_text)
df_val["Preview"] = df_val["Preview"].apply(normalize_text)

In [12]:
df_train["text"] = df_train["Subject"] + df_train["Preview"]

df_val["text"] = df_val["Subject"] + df_val["Preview"]

In [13]:
df_train["Clasificación"] = df_train["Clasificación"].apply(lambda x: 0 if x == "otros" else 1)

In [14]:
df_train["Clasificación"].drop_duplicates()

0    0
1    1
Name: Clasificación, dtype: int64

In [15]:
cv = CountVectorizer()
X_train = cv.fit_transform(df_train["text"])

X_val = cv.transform(df_val["text"])

In [16]:
# Aplicar un TfidfTransformer
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
# X_test = tfidf.fit_transform(X_test)

X_val = tfidf.transform(X_val)

In [17]:
svm_classifier = SVC(kernel='linear')
df_clasificado_svm = cross_val_predict(svm_classifier, X_train, df_train["Clasificación"], cv=3)
confusion_matrix(df_train["Clasificación"], df_clasificado_svm)

array([[ 436,   17],
       [  15, 1175]], dtype=int64)

In [18]:
precision_score(df_train["Clasificación"].to_numpy(), df_clasificado_svm, pos_label=1)

0.985738255033557

In [19]:
svm_classifier.fit(X_train, df_train["Clasificación"])
y_pred = svm_classifier.predict(X_val)
# Agregar la columna clasificación al archivo de validación
df_val["clasificacion"] = y_pred



#df_val.to_excel(r'C:\Users\efreay\Documents\Academia\nuevos_correos.xlsx')

In [21]:
# Definimos las expresiones regulares
numeros_regex = r'\b\d{10,11}\b'
tipologia_regex = r'\b(ahorros|aho|ahorro|corriente|cte|corrientes)\b'

# Creamos las nuevas columnas
df_val['numero'] = df_val['Subject'].str.findall(numeros_regex).apply(' '.join) + df_val['Preview'].str.findall(numeros_regex).apply(' '.join)
df_val['tipo'] = df_val['Subject'].str.findall(tipologia_regex).apply(' '.join) + df_val['Preview'].str.findall(tipologia_regex).apply(' '.join)

# Limpiamos los números de cuenta y NIT
df_val['numero'] = df_val['numero'].str.replace('[-.\s]', '')

# Normalizamos los tipos de cuenta
df_val['tipo'] = df_val['tipo'].replace({'aho': 'Ahorros', 'ahorro': 'Ahorros', 'cte': 'Corriente', 'corrientes': 'Corriente'})

In [23]:
def get_nit(df_val):
  """
  Función para extraer los NIT de las columnas "Subject" y "Preview".

  Parámetros:
    df_certificados: Data frame con las columnas "Subject" y "Preview".

  Devuelve:
    Data frame con una columna adicional llamada "nit".
  """

  # Expresión regular para buscar los NIT en las columnas "Subject" y "Preview".
  regex = re.compile(r"(Nit\.?|NIT\.?|nit\.?)\s+(\d+)")

  # Extraer los NIT de las columnas "subject" y "preview".
  df_val["nit"] = df_val.apply(lambda row: regex.findall(str(row["Subject"])) + regex.findall(str(row["Preview"])), axis=1)

  # Convertir la lista de tuplas a una lista de cadenas.
  df_val["nit"] = df_val["nit"].apply(lambda x: ["".join(tup) for tup in x])

  # Convertir la lista de cadenas a una cadena única.
  df_val["nit"] = df_val["nit"].apply(lambda x: ', '.join(x) if x else np.nan)

  # Devolver el data frame con la columna "nit".
  return df_val

df_val = get_nit(df_val.copy())

In [25]:
df_val.sample(5)

,Folder Path,Subject,DateTimeSent,DateTimeReceived,Preview,text,clasificacion,numero,tipo,nit
3188,\Gestión Marce\,solicitud certificaciones bancolombia octubre,2023-10-06 15:43:26,2023-10-06 15:43:35,cuidado: este correo es externo al grupo banco...,solicitud certificaciones bancolombia octubrec...,1,,,NaN
4704,\Gestión Marce\,re: solicitud referencia bancaria cuenta de ah...,2023-10-03 11:50:55,2023-10-03 11:50:58,equipo_x000d_\nbuenas tardes . espero se encue...,re: solicitud referencia bancaria cuenta de ah...,1,,ahorros,NaN
1449,\Gestión Marce\,re: solicitud certificado bancario 99102 - pa ...,2023-10-12 17:15:59,2023-10-12 17:16:02,hola buenas tardes_x000d_\nagradecemos su vali...,re: solicitud certificado bancario 99102 - pa ...,1,,,NaN
223,\Gestión Marce\,re: confirmaciones auditoria pwc,2023-10-20 09:14:58,2023-10-20 09:15:02,@serviempresarial,re: confirmaciones auditoria pwc@serviempresarial,0,,,NaN
1103,\Gestión Marce\,certificacion bancarias - sempertex de colombia,2023-10-17 10:42:56,2023-10-17 10:43:07,cuidado: este correo es externo al grupo banco...,certificacion bancarias - sempertex de colombi...,1,,Ahorros,NaN


In [28]:
df_val.to_excel(r'C:\Users\efreay\Documents\Academia\newdf.xlsx', index=False)   